## Imports

In [174]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate

from nltk.tokenize import RegexpTokenizer

import pickle 

## EDA

In [175]:
movies = pd.read_csv('./ml-latest-small/movies.csv')

In [176]:
link = pd.read_csv('./ml-latest-small/links.csv')

In [177]:
rating = pd.read_csv('./ml-latest-small/ratings.csv')

In [178]:
tags = pd.read_csv('./ml-latest-small/tags.csv')

In [179]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [180]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [181]:
movies.title.value_counts()

Eros (2004)                                                           2
War of the Worlds (2005)                                              2
Confessions of a Dangerous Mind (2002)                                2
Emma (1996)                                                           2
Saturn 3 (1980)                                                       2
                                                                     ..
Woman in the Dunes (Suna no onna) (1964)                              1
Toy Story (1995)                                                      1
Saragossa Manuscript, The (Rekopis znaleziony w Saragossie) (1965)    1
Five Senses, The (1999)                                               1
Hello, Dolly! (1969)                                                  1
Name: title, Length: 9737, dtype: int64

In [182]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [183]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [184]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [185]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [186]:
#Check length of DataFrame

len(movies)

9737

In [187]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [188]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [189]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [190]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [191]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [192]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [193]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [194]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [195]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [196]:
rating.duplicated().sum()

0

In [197]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [198]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [199]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [200]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [201]:
movie_rating.dropna(inplace=True)

In [202]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [203]:
movie_rating['userId'].nunique()

610

In [204]:
stats = movie_rating.filter(['rating', 'timestamp']).describe()
stats

,rating
count,100813.000000
mean,3.501557
std,1.042494
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [205]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,"action,animation,comedy,fantasy",Black Butler: Book of the Atlantic,2017,184.0,4.0,2018-09-16 14:44:42
100844,193583,"animation,comedy,fantasy",No Game No Life: Zero,2017,184.0,3.5,2018-09-16 14:52:25
100845,193585,drama,Flint,2017,184.0,3.5,2018-09-16 14:56:45
100846,193587,"action,animation",Bungo Stray Dogs: Dead Apple,2018,184.0,3.5,2018-09-16 15:00:21


In [207]:
movie_rating['genres'] = movie_rating['genres'].to_list()

In [208]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
movie_rating['genres'] = [tokenizer.tokenize(token) for token in movie_rating['genres']]

## Split

In [79]:
train, test = train_test_split(movie_rating)

In [80]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [81]:
test_data = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)

## Tuning

### KNNBasics

In [85]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.6s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.0s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   28.8s finished


In [86]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [87]:
base_model.best_estimator

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x7fc7d99384f0>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x7fc7f0c7beb0>}

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [88]:
#instantiate KNN model 
model1=KNNBasic(k=10, min_k=5)


In [89]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9566  0.9586  0.9585  0.9574  0.9582  0.9579  0.0008  
MAE (testset)     0.7322  0.7312  0.7340  0.7317  0.7344  0.7327  0.0013  
Fit time          0.04    0.04    0.04    0.04    0.04    0.04    0.00    
Test time         0.50    0.42    0.45    0.43    0.42    0.44    0.03    


{'test_rmse': array([0.95662226, 0.9586244 , 0.95847623, 0.95736453, 0.95821985]),
 'test_mae': array([0.73216068, 0.73124745, 0.73402863, 0.73171831, 0.73436309]),
 'fit_time': (0.04146003723144531,
  0.043396949768066406,
  0.04292130470275879,
  0.042973995208740234,
  0.04100513458251953),
 'test_time': (0.4954948425292969,
  0.42408299446105957,
  0.4539670944213867,
  0.43243908882141113,
  0.4184870719909668)}

In [90]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=2.5954781702914445, details={'actual_k': 10, 'was_impossible': False})

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 2.5 give or take our RSME which is .95.

## Models

### SVD

In [91]:
model2 = SVD()

In [92]:
model2

In [93]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8733  0.8849  0.8851  0.8905  0.8850  0.8838  0.0057  
MAE (testset)     0.6710  0.6829  0.6808  0.6851  0.6825  0.6804  0.0049  
Fit time          3.08    3.07    3.07    3.12    3.20    3.11    0.05    
Test time         0.15    0.06    0.11    0.07    0.06    0.09    0.03    


{'test_rmse': array([0.87326247, 0.88493798, 0.88511106, 0.89052024, 0.88503285]),
 'test_mae': array([0.67098726, 0.68287995, 0.68078858, 0.68507058, 0.68251225]),
 'fit_time': (3.081982135772705,
  3.0685839653015137,
  3.073172092437744,
  3.117435932159424,
  3.199558734893799),
 'test_time': (0.14911890029907227,
  0.06330490112304688,
  0.10545897483825684,
  0.0747988224029541,
  0.062090158462524414)}

In [94]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  5.2min finished


In [95]:
gs_model2.best_params

{'rmse': {'n_factors': 50, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2},
 'mae': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}}

In [96]:
cross_validate(SVD(n_factors=10,n_epochs=15,lr_all=0.01,reg_all=0.2), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8678  0.8757  0.8793  0.8797  0.8852  0.8775  0.0057  
MAE (testset)     0.6695  0.6737  0.6782  0.6779  0.6810  0.6761  0.0040  
Fit time          0.68    0.70    0.67    0.69    0.68    0.68    0.01    
Test time         0.06    0.14    0.06    0.06    0.06    0.08    0.03    


{'test_rmse': array([0.86781303, 0.87567133, 0.87932099, 0.87974803, 0.88519124]),
 'test_mae': array([0.66945194, 0.67373658, 0.67817905, 0.67793459, 0.68101278]),
 'fit_time': (0.6791110038757324,
  0.699415922164917,
  0.6696891784667969,
  0.6889510154724121,
  0.6825358867645264),
 'test_time': (0.06285405158996582,
  0.1411600112915039,
  0.06126713752746582,
  0.061760902404785156,
  0.06254005432128906)}

In [97]:
param_grid = {'n_factors':[5,10,15],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  9.9min finished


In [98]:
gs_model2.best_params

{'rmse': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1},
 'mae': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1}}

In [99]:
cross_validate(SVD(n_factors=5,n_epochs=15,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8751  0.8742  0.8765  0.8758  0.8729  0.8749  0.0013  
MAE (testset)     0.6723  0.6766  0.6753  0.6706  0.6708  0.6731  0.0024  
Fit time          0.63    0.57    0.58    0.69    0.63    0.62    0.04    
Test time         0.06    0.11    0.06    0.06    0.07    0.07    0.02    


{'test_rmse': array([0.87510021, 0.87423531, 0.87647316, 0.8757893 , 0.87288297]),
 'test_mae': array([0.67225328, 0.67662827, 0.67530597, 0.67059179, 0.67081036]),
 'fit_time': (0.6305809020996094,
  0.5681240558624268,
  0.5798242092132568,
  0.6912031173706055,
  0.6298172473907471),
 'test_time': (0.06019425392150879,
  0.11260795593261719,
  0.060318946838378906,
  0.060355186462402344,
  0.06863784790039062)}

### NMF

In [100]:
model3 = NMF()

In [101]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9311  0.9409  0.9359  0.9529  0.9431  0.9408  0.0074  
MAE (testset)     0.7164  0.7229  0.7176  0.7304  0.7231  0.7221  0.0049  
Fit time          3.26    3.28    3.41    3.26    3.23    3.29    0.06    
Test time         0.05    0.10    0.05    0.05    0.10    0.07    0.02    


{'test_rmse': array([0.9310605 , 0.94090828, 0.93586938, 0.95291381, 0.94306527]),
 'test_mae': array([0.71644085, 0.72288764, 0.71763757, 0.73037795, 0.72309408]),
 'fit_time': (3.2576072216033936,
  3.2819290161132812,
  3.4126431941986084,
  3.2604942321777344,
  3.232913017272949),
 'test_time': (0.05363583564758301,
  0.09865593910217285,
  0.052152156829833984,
  0.05205488204956055,
  0.1030580997467041)}

In [102]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    6.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  2.8min finished


In [103]:
gs_model3.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 50},
 'mae': {'n_factors': 20, 'n_epochs': 50}}

In [104]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9699  0.9793  0.9757  0.9709  0.9806  0.9753  0.0043  
MAE (testset)     0.7353  0.7358  0.7380  0.7371  0.7420  0.7377  0.0024  
Fit time          1.41    1.40    1.42    1.42    1.43    1.42    0.01    
Test time         0.05    0.11    0.05    0.11    0.06    0.08    0.03    


{'test_rmse': array([0.96993939, 0.9793044 , 0.97574823, 0.97092178, 0.98058092]),
 'test_mae': array([0.73532161, 0.73580918, 0.73798238, 0.73714762, 0.74204795]),
 'fit_time': (1.4068889617919922,
  1.400604009628296,
  1.4214591979980469,
  1.4241881370544434,
  1.428539752960205),
 'test_time': (0.05293607711791992,
  0.11087679862976074,
  0.05334877967834473,
  0.10718297958374023,
  0.0592350959777832)}

In [105]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    3.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   49.3s finished


In [106]:
gs_model3.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 25},
 'mae': {'n_factors': 10, 'n_epochs': 25}}

## Deployment 

In [234]:
train['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(train.userId[10], x).est)
train.sort_values(by='est_rating_user10', ascending=False, inplace=True)

In [235]:
train

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
27780,1250,"adventure,drama,war","Bridge on the River Kwai, The",1957,334.0,3.5,2008-10-31 18:28:51,5.0
96095,104879,"drama,mystery,thriller",Prisoners,2013,279.0,5.0,2017-09-26 03:00:11,5.0
26436,1217,"drama,war",Ran,1985,74.0,4.0,2008-04-06 17:12:15,5.0
41101,2239,"comedy,drama",Swept Away,Travolti da un insolito destino nell'azzurro m...,587.0,5.0,2000-03-15 17:05:07,5.0
62202,4437,horror,Suspiria,1977,606.0,4.0,2007-02-18 01:25:48,5.0
...,...,...,...,...,...,...,...,...
71590,6554,"comedy,musical",Garage Days,2002,414.0,2.0,2004-11-09 06:07:41,1.0
85152,53453,"action,adventure,fantasy,sci-fi",Starcrash,a.k.a. Star Crash (1978,448.0,0.5,2017-04-16 18:10:40,1.0
100051,161830,"drama,horror,thriller",Body,2015,448.0,1.0,2016-08-29 17:50:10,1.0
48545,2828,"children,comedy",Dudley Do-Right,1999,292.0,1.0,2002-03-06 00:33:21,1.0


In [230]:
user = input('userId: ')
genre = input('What genres are you interested in? ')
num_recs = input('How many recomendations would you like? ')

userId: 10
What genres are you interested in? drama
How many recomendations would you like? 10


In [236]:
movies['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(uid=user, iid=x).est)
movies.sort_values(by='est_rating_user10', ascending=False, inplace=True) 

In [231]:
genre

'drama'

In [232]:
genre_df = train[train['genres'].str.contains(genre)]

In [233]:
genre_df

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
27780,1250,"adventure,drama,war","Bridge on the River Kwai, The",1957,334.0,3.5,2008-10-31 18:28:51,3.502769
71799,6620,"comedy,drama",American Splendor,2003,610.0,4.0,2017-05-03 21:02:20,3.502769
71783,6620,"comedy,drama",American Splendor,2003,182.0,4.0,2005-05-22 10:04:09,3.502769
71784,6620,"comedy,drama",American Splendor,2003,187.0,5.0,2006-10-26 08:01:25,3.502769
71782,6620,"comedy,drama",American Splendor,2003,91.0,3.5,2005-04-05 14:59:15,3.502769
...,...,...,...,...,...,...,...,...
24411,1184,"comedy,drama",Mediterraneo,1991,606.0,4.5,2007-02-12 23:39:37,3.502769
24408,1184,"comedy,drama",Mediterraneo,1991,391.0,4.0,2002-09-03 05:25:26,3.502769
24407,1184,"comedy,drama",Mediterraneo,1991,202.0,4.0,2000-11-22 20:21:42,3.502769
22493,1046,"drama,romance",Beautiful Thing,1996,596.0,4.0,2018-09-01 20:12:24,3.502769


In [226]:
for x in train['genre']:
    if genre in x:
        print(x)

KeyError: 'genre'

In [224]:
if genre in train['genres']:
    #print(genre)
    print(train['genres'])
    

In [237]:
train #['Title'].head(int(num_recs))

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
27780,1250,"adventure,drama,war","Bridge on the River Kwai, The",1957,334.0,3.5,2008-10-31 18:28:51,5.0
96095,104879,"drama,mystery,thriller",Prisoners,2013,279.0,5.0,2017-09-26 03:00:11,5.0
26436,1217,"drama,war",Ran,1985,74.0,4.0,2008-04-06 17:12:15,5.0
41101,2239,"comedy,drama",Swept Away,Travolti da un insolito destino nell'azzurro m...,587.0,5.0,2000-03-15 17:05:07,5.0
62202,4437,horror,Suspiria,1977,606.0,4.0,2007-02-18 01:25:48,5.0
...,...,...,...,...,...,...,...,...
71590,6554,"comedy,musical",Garage Days,2002,414.0,2.0,2004-11-09 06:07:41,1.0
85152,53453,"action,adventure,fantasy,sci-fi",Starcrash,a.k.a. Star Crash (1978,448.0,0.5,2017-04-16 18:10:40,1.0
100051,161830,"drama,horror,thriller",Body,2015,448.0,1.0,2016-08-29 17:50:10,1.0
48545,2828,"children,comedy",Dudley Do-Right,1999,292.0,1.0,2002-03-06 00:33:21,1.0


In [238]:
movies #['Title'].head(int(num_recs))

,movieId,genres,Title,year_released,est_rating_user10
1,2,Adventure|Children|Fantasy,Jumanji,1995,3.502769
36,40,Drama,"Cry, the Beloved Country",1995,3.502769
6,7,Comedy|Romance,Sabrina,1995,3.502769
7,8,Adventure|Children,Tom and Huck,1995,3.502769
8,9,Action,Sudden Death,1995,3.502769
...,...,...,...,...,...
9727,190213,Drama,John From,2015,NaN
9729,190219,Animation,Bunny,1998,NaN
9733,193567,Animation|Drama,anohana: The Flower We Saw That Day - The Movie,2013,NaN
9736,193579,Documentary,Jon Stewart Has Left the Building,2015,NaN


In [ ]:
movies[]